In [1]:

    import requests
    from bs4 import BeautifulSoup

    base_url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code
    =179159&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}"
    reviews = []
    for n in range(10):
        url = base_url.format(n+1)
        resp = requests.get(url)
        html = BeautifulSoup(resp.content, 'html.parser')

        score_result = html.find('div', {'class': 'score_result'})
        score_result2 = html.find('div', {'class': 'star_score'})
    
    
        lis = score_result.findAll('li')
    
        for li in lis :
            review_text = li.find('p').getText()
            star = score_result2.getText()
            comment = review_text.strip().replace("\t","").replace("\n","").replace("\r","")
            star = int(star)
            label = 0 if star < 5 else 1
            reviews.append([label, comment])
    training_sentences = [[], []]    
    for review in reviews:
        label = review[0]
        comment = review[1]
        
        training_sentences[0].append(label)
        training_sentences[1].append(comment)


ConnectionError: HTTPSConnectionPool(host='movie.naver.com', port=443): Max retries exceeded with url: /movie/bi/mi/pointWriteFormList.nhn?code=179159&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002E186BB25F8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [1]:
score = training_sentences[0]
text = training_sentences[1]

len(score)
len(text)

NameError: name 'training_sentences' is not defined

In [2]:
import pandas as pd

df = pd.DataFrame([score, text]).T
df.columns = ['score', 'text']
df

NameError: name 'score' is not defined

In [1]:
df.to_csv('text.csv')

NameError: name 'df' is not defined

In [70]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(text, score, test_size = 0.2, random_state=0)

In [71]:
len(train_x), len(train_y)

(4376, 4376)

In [72]:
len(test_x),len(test_y)

(1094, 1094)

In [75]:
train_x

["연기며 표정이며 최고에요 긴장하면서 봤네요!'",
 '이런 영화를 언제까지 봐야 할까. 이런 영화가 기생충 나온 한국에서 박스오피스 1위인 것도 너무 한심하다',
 '감ㅅ하합니다 가 그의 첫대사',
 '올해 액션한국영화중 젤 낳은듯~^^',
 '군더더기  없이 깔끔하다',
 '연기력 별로, 이야기 별로, 공감도 안되고 허성태는 왜 나온거임??조조로 봤지만 그냥 안보는게 나음',
 '외톨이 캐릭터가 이 영화의 유일한 오점...',
 '너무 빠른 전개가 아쉬웠지만 정말 재밌게본 영화,3편이 기대되는 영화였다',
 '1편도 재밌게봐서 기대했었는데 실망감이 없어서 너무 기쁘네요 ㅠ 재밌게봤습니다아아',
 '',
 '연차내고 봤습니다. 재밌네요',
 '너무 재밌어요...액션씬 진짜 대박...바둑은 거들뿐!',
 '관람객평점 저정도만큼되는 영화는솔직히아니고 그냥 볼만한정도 근데 너무 말도안되는게 많아서 오그라듬',
 '알바들이 진짜 열일하는구나 이런 망작 쓰레기 영화 평점 주는거 보면망작의 기운이 영화 5분만에 느껴질만큼 상영 시간이 괴롭고 힘들었다이런걸 만드는 제작진분들 진짜 대단하고 투자자들도 대단하다한국영화 작년부터 유독 올...',
 '관람객잼나네요. 연출을 잘한거 같아요 잘 봤어요^^',
 '신의 한 수... 심야관람... 끝까지 몰입해서 보았음',
 '저는 신의한수1보다 더ㅓ재밌어요',
 '기대없이 봐서 그런지 돈 안아까움 강추',
 '너무 재밌었어요권상우 너무 멋짐 ㅠㅠ',
 '긴장감 끝까지 있고. 배우들 연기 좋아요. 약간 잔인한거 빼곤 다 좋아요. 액션 영화라해도 될 정도로 끝내줍니다. 권상우 인생작 나왔네요.',
 '',
 '연출 연기 전부다 좋았지만 극중에 외톨이라는 캐릭터의 비중이 많이 없다는게 안타까움 나름 주인공이랑 대결구도가 되어있는 캐릭터인데ㅡㅡ',
 '보는동안  볼만하다보고나면  ..1편과  묶어보면  볼만함',
 '너무 잔인하다... 여자들이 보기엔 잔인하고 남자들은 뭐...15세 관람가 치고 폭력성이 넘 높다..',
 '진짜 안보면 

In [76]:
from konlpy.tag import Twitter
twitter = Twitter()
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(tokenizer= twitter.morphs, ngram_range=(1,2), min_df= 3, max_df = 0.9) 
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
tfv_train_x


<4376x5892 sparse matrix of type '<class 'numpy.float64'>'
	with 82420 stored elements in Compressed Sparse Row format>

In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

clf = LogisticRegression(random_state = 0)
params = {'C' : [1,3,5,7,9]}
grid_cv = GridSearchCV(clf, param_grid = params, cv=4, scoring = 'accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.5s finished


GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None, param_grid={'C': [1, 3, 5, 7, 9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=1)

In [81]:
grid_cv.best_params_

{'C': 5}

In [83]:
grid_cv.best_score_

0.8612888482632541

In [90]:
tfv_test_x = tfv.transform(test_x)
grid_cv.best_estimator_.score(tfv_test_x, test_y)

0.8775137111517367

In [108]:
a = ['아 너무 재미있어요', '핵노잼','핵노잼','노잼', '영화가 지루해요', '보지마삼']
my_review = tfv.transform(a)
grid_cv.best_estimator_.predict(my_review)

array([1, 0, 0, 0, 1, 1])